In [ ]:
from collections import Counter
from pathlib import Path
import pickle

import torch
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import manifold
import dgl
from tqdm.auto import tqdm

from train import MultiSAGEModel, PinSAGECollator

In [ ]:
datapath = Path('..', 'data')

In [ ]:
with open(datapath / 'dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [ ]:
with open(datapath / 'cuisine.pkl', 'rb') as f:
    cuisine = pickle.load(f)

with open(datapath / 'ingredient.pkl', 'rb') as f:
    ingredient = pickle.load(f)

In [ ]:
graph = dgl.load_graphs(str(datapath / 'graph.bin'))[0][0]
graph

In [ ]:
model = MultiSAGEModel(
    graph,
    'ingredient',
    'recipe',
    'cuisine',
    256,
    2,
    3,
)
model.load_state_dict(torch.load(
    Path('..', 'saved', '39_3model_state_dict.pt'),
    map_location=torch.device('cpu')
))

In [ ]:
ingredient = ingredient.detach().numpy().astype('float')

In [ ]:
h_item = ingredient

In [ ]:
def ingredient_scores(ingredients):
    scores = []
    for new_ingredient in range(graph.number_of_nodes('ingredient')):
        score = 0
        for ingredient in ingredients:
            score += (
                np.dot(h_item[ingredient], h_item[new_ingredient])
                + model.node_scorer.bias[ingredient]
                + model.node_scorer.bias[new_ingredient])
        scores.append(score)
    return scores

In [ ]:
all_scores = []
for recipe in enumerate(dataset['X_cmp_val']):
    all_scores.append(ingredient_scores(recipe))

In [ ]:
dataset['y_cmp_val']